## Colab
Run the following part only if you opened this notebook in Google Colab.

<a href="https://colab.research.google.com/github/davide-gurrieri/plants-classifier/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## RUN FROM HERE IF YOU WANT TO RUN IT INDEPENDENTLY FROM THE GITHUB
## ! BE AWARE THAT YOU NEED TO HAVE TE COPY OF THE REPOSITORY AND OF AN2DL COURSE IN YOUR DRIVE FOLDER !

# Import the data from the drive
from google.colab import drive
drive.mount('/content/drive')
%cp -r "drive/MyDrive/plants-classifier" "plants-classifier/"

Mounted at /content/drive


In [2]:
# Copy the data from the drive to the local repository folder
%cp "drive/MyDrive/[2023-2024] AN2DL/Homework 1/public_data.zip" "plants-classifier/data/"
# Unzip the data
!unzip plants-classifier/data/public_data.zip -d plants-classifier/data/
# Remove the zip file
!rm plants-classifier/data/public_data.zip
%cd plants-classifier/

Archive:  plants-classifier/data/public_data.zip
  inflating: plants-classifier/data/public_data.npz  
/content/plants-classifier


Now you are ready to run the notebook. You are inside the folder `plants-classifier`.

## Import libraries

In [3]:
!pip install keras-tuner
import kerastuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 3.4 MB/s eta 0:00:00


<ipython-input-3-f4fc69484a61>:2: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner


In [4]:
# Custom modules
from imports import *
import utils
# import models.QuasiVGG9 as QuasiVGG9
# import models.QuasiVGG9Flatten as QuasiVGG9Flatten
# import models.Xception as Xception
import models.ConvNeXtBaseKerasTuner as ConvNeXtBaseKerasTuner
# import model

2.14.0


## Load, inspect and process data

In [5]:
X_train_val_with_out, y_train_val_with_out, X_train_val, y_train_val, labels, X_out, y_out, shrek, trol = utils.data_processing()
print()
print("Shape of X_train_val: ", X_train_val.shape)
print("Shape of y_train_val: ", y_train_val.shape)

The dataset without outliers contains 5004 images of plants, 3101 healthy and 1903 unhealthy.
The ratio of the healthy plants over the total is 0.62.
The ratio of the healthy plants over the total considering also outliers is 0.62.
Each image has shape (96, 96, 3).
The labels encoding is: {0: 'healthy', 1: 'unhealthy'}.

Shape of X_train_val:  (5004, 96, 96, 3)
Shape of y_train_val:  (5004, 1)


In [ ]:
# unique_values, indices = np.unique(X_train_val, return_inverse=True, axis=0)
# duplicate_indices = np.where(np.bincount(indices) > 1)[0]
# X_repeated = X_train_val[duplicate_indices]

In [ ]:
utils.plot_images(
    X_train_val,
    y_train_val,
    num_img=100,
    show=True,
    save=False,
    name="images.pdf",
)

In [ ]:
utils.plot_images(
    X_out,
    y_out,
    num_img=100,
    show=True,
    save=False,
    name="outliers.pdf",
)

In [ ]:
# np.save('shrek.npy', shrek)
# np.save('trol.npy', trol)

In [6]:
# Split data into training and validation sets, maintaining class distribution
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, random_state=SEED, test_size=0.2, stratify=y_train_val)

# Print the shapes of the resulting datasets
print("Training Data Shape:", X_train.shape)
print("Training Label Shape:", y_train.shape)
print("Validation Data Shape:", X_val.shape)
print("Validation Label Shape:", y_val.shape)

Training Data Shape: (4003, 96, 96, 3)
Training Label Shape: (4003, 1)
Validation Data Shape: (1001, 96, 96, 3)
Validation Label Shape: (1001, 1)


## Model definition, building and training

In [9]:
input_shape = X_train.shape[1:]  # Input shape for the model
output_shape = y_train.shape[1]  # Output shape for the model
print("Input Shape:", input_shape)
print("Output Shape:", output_shape)

Input Shape: (96, 96, 3)
Output Shape: 1


In [6]:
# import keras_tuner

In [10]:
build_param_1 = {
    "input_shape": (96, 96, 3),
    "output_shape": 1,
}

compile_param_1 = {
    "loss": tfk.losses.BinaryCrossentropy(),
    "optimizer": tfk.optimizers.Adam(learning_rate=1e-4),
    "metrics": ["accuracy"],
}

fit_param_1 = {
    "batch_size": 32,
    "epochs": 200,
    "callbacks": [
        tfk.callbacks.EarlyStopping(
            monitor="val_accuracy",
            patience=20,
            mode="max",
            restore_best_weights=True,
        )
    ],
}

In [11]:
def build(hp):

        tf.random.set_seed(42)

        augmentation = tf.keras.Sequential(
            [
                tfkl.RandomFlip(mode="horizontal"),
                tfkl.RandomFlip(mode="vertical"),
                tfkl.RandomRotation(factor=0.25),
                # tfkl.RandomCrop(height=64, width=64),
                tfkl.RandomZoom(height_factor=0.2),
                # tfkl.RandomContrast(factor=0.8),
            ],
            name="preprocessing",
        )

        relu_init = tfk.initializers.HeUniform(seed=42)

        input_layer = tfkl.Input(shape=build_param_1["input_shape"], name="Input")

        augmentation_layer = augmentation(input_layer)

        # Build the ResNet50
        ConvNeXtBase=tfk.applications.ConvNeXtBase(
            include_top=False,
            include_preprocessing=True,
            weights="imagenet",
            input_tensor=None,
            input_shape=build_param_1["input_shape"],
            classes =2,
            pooling="avg",
        )

        x = ConvNeXtBase(augmentation_layer)

        x = tfkl.Dropout(hp.Float("dropout_1", min_value=0.3, max_value=0.6, step=0.1))(x)

        x = tfkl.Dense(
            units=hp.Int("dense_units_1", min_value=512, max_value=2048, step=256),
            activation="relu",
            kernel_initializer=relu_init,
        )(x)

        x = tfkl.Dropout(hp.Float("dropout_2", min_value=0.2, max_value=0.5, step=0.1))(x)

        x = tfkl.Dense(
            units=hp.Int("dense_units_2", min_value=256, max_value=1024, step=128),
            activation="relu",
            kernel_initializer=relu_init,
        )(x)

        x = tfkl.Dropout(hp.Float("dropout_3", min_value=0.1, max_value=0.4, step=0.1))(x)

        x = tfkl.Dense(
            units=hp.Int("dense_units_3", min_value=128, max_value=512, step=64),
            activation="relu",
            kernel_initializer=relu_init,
        )(x)

        x = tfkl.Dropout(hp.Float("dropout_4", min_value=0.1, max_value=0.3, step=0.1))(x)

        x = tfkl.Dense(
            units=56,
            activation="relu",
            kernel_initializer=relu_init,
        )(x)

        x = tfkl.Dropout(hp.Float("dropout_5", min_value=0, max_value=0.2, step=0.1))(x)

        output_layer = tfkl.Dense(
            units=build_param_1["output_shape"],
            activation="sigmoid",
            kernel_initializer=tfk.initializers.GlorotUniform(seed=42),
            name="Output",
        )(x)

        # Connect input and output through the Model class
        model = tfk.Model(inputs=input_layer, outputs=output_layer, name="ConvNeXtBase")

        model.compile(optimizer=compile_param_1["optimizer"], loss=compile_param_1["loss"], metrics=compile_param_1["metrics"])

        return model


In [12]:
# Instantiate the tuner
tuner = kerastuner.RandomSearch(
    build,
    objective="val_accuracy",
    max_trials=5,
    # directory="tuner_dir",
    # project_name="ConvNeXtBase_tuning",
)

# Perform the hyperparameter search
tuner.search(X_train, y_train, epochs=ConvNeXtBaseKerasTuner.fit_param_1["epochs"], validation_data=(X_val, y_val))

# Get the best hyperparameters
# best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Build the final model with the best hyperparameters
best_model = tuner.hypermodel.build()[0]


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
0.5               |0.5               |dropout_1
1792              |1792              |dense_units_1
0.2               |0.2               |dropout_2
896               |896               |dense_units_2
0.2               |0.2               |dropout_3
448               |448               |dense_units_3
0.1               |0.1               |dropout_4
0                 |0                 |dropout_5

Epoch 1/200
126/126 [==============================] - 175s 681ms/step - loss: 0.6215 - accuracy: 0.6835 - val_loss: 0.6786 - val_accuracy: 0.7353
Epoch 2/200
126/126 [==============================] - 61s 482ms/step - loss: 0.4241 - accuracy: 0.8136 - val_loss: 0.2586 - val_accuracy: 0.9061
Epoch 3/200
126/126 [==============================] - 51s 406ms/step - loss: 0.2874 - accuracy: 0.8831 - val_loss: 0.2297 - val_accuracy: 0.8971
Epoch 4/200
126/126 [==============================] - 58s 458ms/step - loss: 0.2260

KeyboardInterrupt: ignored

In [ ]:
# create an object of the custom model class
model_obj = ConvNeXtBaseAdaptiveLR.ConvNeXtBaseAdaptiveLR("ConvNeXtBaseAdaptiveLR", ConvNeXtBaseAdaptiveLR.build_param_1, ConvNeXtBaseAdaptiveLR.compile_param_1, ConvNeXtBaseAdaptiveLR.fit_param_1)

In [ ]:
# build and compile the model
model_obj.build()
model_obj.compile()
model_obj.model.summary()

### Train the model

In [ ]:
model_obj.train_val(X_train, y_train, X_val, y_val, one_hot=False)

In [ ]:
# save the model
model_obj.save_model()

In [ ]:
# plot the training and validation loss and accuracy
model_obj.plot_history()

In [ ]:
# prediction
model_obj.evaluate(X_val, y_val)

Accuracy: 0.9351
Precision: 0.9158
Recall: 0.9134
Confusion matrix:
[[588  32]
 [ 33 348]]


In [ ]:
# FINE TUNING large

# ft_model = model_obj.model
# ft_model = tfk.models.load_model("../drive/MyDrive/VGG16")

for i, layer in enumerate(model_obj.get_layer('convnext_large').layers):
   print(i, layer.name, layer.trainable)

for i, layer in enumerate(model_obj.get_layer('convnext_large').layers[270:293]):
  layer.trainable=True
for i, layer in enumerate(ft_model.get_layer('convnext_large').layers):
   print(i, layer.name, layer.trainable)

ft_mmodel_objodel.compile(loss=tfk.losses.BinaryCrossentropy(), optimizer=tfk.optimizers.Adam(1e-5), metrics='accuracy')

ft_history = ft_model.fit(
    x = X_train, # We need to apply the preprocessing thought for the MobileNetV2 network
    y = y_train,
    batch_size = 32,
    epochs = 200,
    validation_data = (X_val, y_val), # We need to apply the preprocessing thought for the MobileNetV2 network
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=10, restore_best_weights=True)]
).history

In [ ]:
%cd /content

/content


In [ ]:
## RUN THIS CELL IF YOU WANT TO SAVE THE TRAINED MODEL IN YOUR DRIVE FOLDER
## then download it from the driveand put the content in the SubmissionModel folder
%cd ..
%cp -r "/content/plants-classifier/saved_models/ConvNeXtBaseAdaptiveLR" "drive/MyDrive/"

/content


In [ ]:
model = tfk.models.load_model('saved_models/ConvNeXtBase')